In [43]:
import tensorflow as tf
from tensorflow.contrib import rnn
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
tf.reset_default_graph()

In [44]:
data_frame = pd.read_csv("/home/oh/data/spy.csv")

In [45]:
timesteps = seq_length = 1
data_dim = 13
hidden_dim = 4
output_dim = 1
learing_rate = 0.0005
iterations = 50000

In [46]:
data_frame["screen"] = data_frame["screen"]/1000
data_frame["Y"] = data_frame["Y"]/1000000
data_frame["seat"] = data_frame["seat"]/10000000

In [47]:
x = data_frame.values
y = data_frame["Y"].values  


In [48]:
dataX = []
dataY = []

In [49]:
for i in range(0, len(y) - seq_length):
    _x = np.copy(x[i:i + seq_length + 1])
    _x[timesteps-2][data_dim-1] = 0
    _x[timesteps-1][data_dim-1] = 0
    _x[timesteps][data_dim-1] = 0
    _y = [y[i + seq_length]]
    dataX.append(_x)
    dataY.append(_y)

In [50]:
train_size = int(len(dataY) * 0.575)
test_size = len(dataY) - train_size 

trainX = np.array(dataX[:train_size])
testX = np.array(dataX[train_size : ])

trainY = np.array(dataY[:train_size])
testY = np.array(dataY[train_size : ])


In [51]:
X = tf.placeholder(tf.float32, [None, seq_length+1, data_dim])
Y = tf.placeholder(tf.float32, [None, 1])

def lstm_cell(): 
    cell = rnn.BasicLSTMCell(hidden_dim, state_is_tuple=True) 
    return cell 


cell = rnn.BasicLSTMCell(num_units=hidden_dim, state_is_tuple=True)
multi_cells = rnn.MultiRNNCell([lstm_cell() for _ in range(5)], state_is_tuple=True)


outputs, _states = tf.nn.dynamic_rnn(multi_cells, X, dtype=tf.float32)

Y_pred = tf.contrib.layers.fully_connected(outputs[:, -1], output_dim, activation_fn=None)

loss = tf.reduce_sum(tf.square(Y_pred - Y))  
train = tf.train.RMSPropOptimizer(learing_rate).minimize(loss)

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

for i in range(iterations):
    _  , cost = sess.run([train ,loss], feed_dict={X: trainX, Y: trainY})
    if (i+1) % (iterations/10) == 0:
        print("[step: {}] loss: {}".format(i+1, cost))


[step: 5000] loss: 2.87370403384557e-05
[step: 10000] loss: 9.92188597592758e-06
[step: 15000] loss: 6.869997378089465e-06
[step: 20000] loss: 5.556919859373011e-06
[step: 25000] loss: 4.912109943688847e-06
[step: 30000] loss: 4.452162556844996e-06
[step: 35000] loss: 4.1043799683393445e-06
[step: 40000] loss: 3.833348728221608e-06
[step: 45000] loss: 3.6959206681785872e-06
[step: 50000] loss: 3.6178723803459434e-06


In [52]:
train_predict = sess.run(Y_pred, feed_dict={X: trainX})
test_predict = sess.run(Y_pred, feed_dict={X: testX})

In [53]:
test_predict

array([[ 2.4893135e-04],
       [ 8.6676888e-04],
       [ 1.1728425e-03],
       [ 1.6960781e-04],
       [ 3.7280377e-04],
       [ 2.7086399e-04],
       [ 7.0361421e-05],
       [ 1.5681144e-04],
       [ 2.8990395e-04],
       [ 2.3887306e-04],
       [-1.3585575e-04],
       [-2.3088418e-05],
       [-5.8641657e-05],
       [-1.4171936e-04],
       [-1.0760594e-04],
       [-6.9381669e-05],
       [-1.0225829e-04]], dtype=float32)

In [54]:
dataframe1=pd.DataFrame(test_predict)
dataframe2=pd.DataFrame(train_predict)

In [55]:
dataframe1.to_csv("/home/oh/train_your wedding.csv",header=False,index=False)

In [56]:
dataframe2.to_csv("/home/oh/pred_your wedding.csv",header=False,index=False)

In [57]:
train_predict

array([[0.01898882],
       [0.02658086],
       [0.04178432],
       [0.03593073],
       [0.01317515],
       [0.01216906],
       [0.00899482],
       [0.00571517],
       [0.0071622 ],
       [0.01018946],
       [0.01100009],
       [0.00357796],
       [0.00364487],
       [0.00292498],
       [0.00166951],
       [0.00214229],
       [0.0032597 ],
       [0.00317747],
       [0.00072695],
       [0.00053563],
       [0.00036073],
       [0.00014764]], dtype=float32)